In [ ]:
MariaDB [(none)]> create database universitas;
Query OK, 1 row affected (0.002 sec)

MariaDB [universitas]> create table mahasiswa (
    -> NPM varchar(5) primary key,
    -> Nama varchar(100) not null,
    -> Alamat text(100),
    -> Id_kelurahan INT,
    -> Jenis_kelamin enum ('L', 'P'),
    -> No_Hp varchar(15),
    -> Kode_prodi INT null );
Query OK, 0 rows affected (0.026 sec)

MariaDB [universitas]> insert into mahasiswa (npm, nama, alamat, id_kelurahan, jenis_kelamin, no_hp, kode_prodi) values
    -> ('12345', 'Falikha Sulfiyah', 'Cirebon', '1', 'P', '081234567891', '1'),
    -> ('12346', 'Budi Sugandhi', 'Bekasi', '109', 'L', '085643433321', '2'),
    -> ('12347', 'Toha Sitohang', 'Medan', '103', 'L', '08989787876', '2'),
    -> ('12348', 'Mega', 'Condong Catur', '107', 'P', '0839303058', '1'),
    -> ('12349', 'Ritsuki', 'Jakarta', '107', 'P', '08132809811', null),
    -> ('12780', 'Anis Baswedan', 'Jakarta', '234', 'L', '09898877663', null),
    -> ('12786', 'Ganjar Pranowo', 'Jawa Tengah', '123', 'L', '087656356622', null);
Query OK, 7 rows affected (0.019 sec)
Records: 7  Duplicates: 0  Warnings: 0

MariaDB [universitas]> CREATE PROCEDURE view_mhs_simple()
    -> BEGIN
    ->     SELECT NPM, nama, no_hp
    ->     FROM mahasiswa;
    -> END //
Query OK, 0 rows affected (0.033 sec)

MariaDB [universitas]> DELIMITER $$
MariaDB [universitas]> CREATE FUNCTION tambah (angka1 INT, angka2 INT) RETURNS INT
    -> BEGIN
    -> RETURN angka1 + angka2;
    -> END; $$
Query OK, 0 rows affected (0.022 sec)

MariaDB [universitas]> create table barang (
    -> id_brg CHAR(4) primary key,
    -> nama_brg varchar(20) not null,
    -> stok INT not null );
Query OK, 0 rows affected (0.034 sec)

MariaDB [universitas]> create table pembelian (
    -> id_pem INT primary key,
    -> id_brg CHAR(4),
    -> jml_beli INT not null,
    -> FOREIGN KEY (id_brg) REFERENCES barang (id_brg) );
Query OK, 0 rows affected (0.018 sec)

MariaDB [universitas]> insert into barang (id_brg, nama_brg, stok) values
    -> ('A10', 'Mouse', 10),
    -> ('A11', 'Keyboard', 15),
    -> ('A12', 'DVD R-W', 10);
Query OK, 3 rows affected (0.018 sec)
Records: 3  Duplicates: 0  Warnings: 0

MariaDB [universitas]> insert into pembelian (id_pem, id_brg, jml_beli) values
    -> (1, 'A10', 5);
Query OK, 1 row affected (0.005 sec)

MariaDB [universitas]> insert into barang values ('A13', 'Modem', 5);
Query OK, 1 row affected (0.016 sec)

MariaDB [universitas]> insert into pembelian (id_pem, id_brg, jml_beli) values
    -> (2, 'A10', 10);
Query OK, 1 row affected (0.016 sec)

MariaDB [universitas]> ALTER TABLE pembelian DROP FOREIGN KEY pembelian_ibfk_1;
Query OK, 0 rows affected (0.030 sec)
Records: 0  Duplicates: 0  Warnings: 0

#TUGAS

MariaDB [universitas]> create table buku (
    -> id_buku INT AUTO_INCREMENT primary key,
    -> judul VARCHAR(100),
    -> penulis VARCHAR(100),
    -> harga DECIMAL(10,2),
    -> stok INT );
Query OK, 0 rows affected (0.034 sec)

MariaDB [universitas]> create table pelanggan (
    -> id_pelanggan INT AUTO_INCREMENT primary key,
    -> nama VARCHAR(100),
    -> total_belanja DECIMAL(10,2) DEFAULT 0,
    -> status_member ENUM('REGULER', 'GOLD', 'PLATINUM') DEFAULT 'REGULER' );
Query OK, 0 rows affected (0.011 sec)

MariaDB [universitas]> create table transaksi (
    -> id_transaksi INT AUTO_INCREMENT primary key,
    -> id_pelanggan INT,
    -> id_buku INT,
    -> jumlah INT,
    -> total_harga DECIMAL(10,2),
    -> tanggal_transaksi DATE,
    -> foreign key (id_pelanggan) references pelanggan(id_pelanggan),
    -> foreign key (id_buku) references buku(id_buku) );
Query OK, 0 rows affected (0.027 sec)

MariaDB [universitas]> DELIMITER //
MariaDB [universitas]> CREATE FUNCTION hitung_diskon(total_belanja DECIMAL(10,2))
    -> RETURNS DECIMAL(5,2)
    -> DETERMINISTIC
    -> BEGIN
    ->     DECLARE diskon DECIMAL(5,2);
    ->     IF total_belanja < 1000000 THEN
    ->         SET diskon = 0.00;
    ->     ELSEIF total_belanja < 5000000 THEN
    ->         SET diskon = 0.05;
    ->     ELSE
    ->         SET diskon = 0.10;
    ->     END IF;
    ->     RETURN diskon;
    -> END //
Query OK, 0 rows affected (0.025 sec)

MariaDB [universitas]> CREATE PROCEDURE tambah_transaksi (
    ->     IN p_id_pelanggan INT,
    ->     IN p_id_buku INT,
    ->     IN p_jumlah INT
    -> )
    -> BEGIN
    ->     DECLARE v_harga DECIMAL(10,2);
    ->     DECLARE v_total_harga DECIMAL(10,2);
    ->     DECLARE v_stok INT;
    ->
    ->     -- Ambil harga dan stok dari buku
    ->     SELECT harga, stok INTO v_harga, v_stok
    ->     FROM buku
    ->     WHERE id_buku = p_id_buku;
    ->
    ->     -- Validasi stok
    ->     IF v_stok < p_jumlah THEN
    ->         SIGNAL SQLSTATE '45000'
    ->         SET MESSAGE_TEXT = 'Stok tidak mencukupi';
    ->     ELSE
    ->         -- Hitung total harga
    ->         SET v_total_harga = v_harga * p_jumlah;
    ->
    ->         -- Update stok buku
    ->         UPDATE buku
    ->         SET stok = stok - p_jumlah
    ->         WHERE id_buku = p_id_buku;
    ->
    ->         -- Insert ke tabel transaksi
    ->         INSERT INTO transaksi(id_pelanggan, id_buku, jumlah, total_harga)
    ->         VALUES(p_id_pelanggan, p_id_buku, p_jumlah, v_total_harga);
    ->
    ->         -- Update total belanja pelanggan
    ->         UPDATE pelanggan
    ->         SET total_belanja = total_belanja + v_total_harga
    ->         WHERE id_pelanggan = p_id_pelanggan;
    ->
    ->         -- Tampilkan pesan sukses
    ->         SELECT 'Transaksi berhasil' AS pesan;
    ->     END IF;
    -> END //
Query OK, 0 rows affected (0.026 sec)

MariaDB [universitas]> DELIMITER $$
MariaDB [universitas]> CREATE TRIGGER trg_update_status_member
    -> AFTER UPDATE ON pelanggan
    -> FOR EACH ROW
    -> BEGIN
    ->     DECLARE new_status ENUM('REGULER', 'GOLD', 'PLATINUM');
    ->     IF NEW.total_belanja >= 5000000 THEN
    ->         SET new_status = 'PLATINUM';
    ->     ELSEIF NEW.total_belanja >= 1000000 THEN
    ->         SET new_status = 'GOLD';
    ->     ELSE
    ->         SET new_status = 'REGULER';
    ->     END IF;
    ->
    ->     IF NEW.status_member <> new_status THEN
    ->         UPDATE pelanggan
    ->         SET status_member = new_status
    ->         WHERE id_pelanggan = NEW.id_pelanggan;
    ->     END IF;
    -> END $$
Query OK, 0 rows affected (0.035 sec)

